<a href="https://colab.research.google.com/github/Akshay-ramagiri/NNDL/blob/main/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##MOUNT GOOGLE DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##IMPORTING REQUIRED PACKAGES

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

##IMPORT DATA

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Assignment2.csv')
data.head()

,Date,Time,Load (kW)
0,01.09.2018,0:00,5551.82208
1,NaN,1:00,4983.17184
2,NaN,2:00,4888.39680
3,NaN,3:00,5072.95872
4,NaN,4:00,5196.25980


In [ ]:
x = []
y = []
for i in range(24, data.shape[0]):
    x.append(data['Load (kW)'].iloc[i-24])
    y.append(data['Load (kW)'].iloc[i])

In [ ]:
data = pd.DataFrame({'Previous Day Load' : x, 'Present Day Load' : y})
data.head()

,Previous Day Load,Present Day Load
0,5551.82208,4931.26380
1,4983.17184,4775.53968
2,4888.39680,4713.81120
3,5072.95872,4689.02628
4,5196.25980,4844.75040


##NORMALIZATION

In [ ]:
Xmax = np.max(data['Previous Day Load'])
Xmin = np.min(data['Previous Day Load'])
Ymax = np.max(data['Present Day Load'])
Ymin = np.min(data['Present Day Load'])
data['Previous Day Load'] = (data['Previous Day Load'] - Xmin) / (Xmax - Xmin)
data['Present Day Load'] = (data['Present Day Load'] - Ymin) / (Ymax - Ymin)
data.head()

,Previous Day Load,Present Day Load
0,0.397877,0.284300
1,0.293800,0.255799
2,0.276454,0.244501
3,0.310234,0.239965
4,0.332801,0.268466


##DATA SPLIT

In [ ]:
x = data['Previous Day Load']
y = data['Present Day Load']
Xtrain, Xtest, Ytrain, Ytest = train_test_split(x, y, test_size=0.1, random_state=101)

##GRADIENT CALCULATION AND MODEL TRAINING

In [ ]:
m = np.random.uniform(-5, 5)
c = np.random.uniform(-5, 5)
eta = 0.01
epochs = 500
vm = 0
vc = 0
v = 0.5

for _ in range(epochs):
    for i in range(Xtrain.shape[0]):
        gM = -1 * (Ytrain.iloc[i] - m * Xtrain.iloc[i] - c) * Xtrain.iloc[i]
        gC = -1 * (Ytrain.iloc[i] - m * Xtrain.iloc[i] - c)
        
        vm = (v * vm) - (eta * gM)
        vc = (v * vc) - (eta * gC)
 
        m += vm
        c += vc

print(f'm = {m}\nc = {c}')

m = 0.7171132537325575
c = 0.12511078784145216


In [ ]:
train_pred = [m * Xtrain.iloc[i] + c for i in range(Xtrain.shape[0])]
test_pred = [m * Xtest.iloc[i] + c for i in range(Xtest.shape[0])]

##DENORMALIZATION

In [ ]:
train_pred = [i * (Ymax - Ymin) + Ymin for i in train_pred]
test_pred = [i * (Ymax - Ymin) + Ymin for i in test_pred]
trainY = [i * (Ymax - Ymin) + Ymin for i in Ytrain]
testY = [i * (Ymax - Ymin) + Ymin for i in Ytest]

##ERROR METRIC CALCULATION
TRAINING DATA

In [ ]:
train_MAE = sum([abs(train_pred[i] - trainY[i]) for i in range(len(trainY))]) / len(trainY)
train_MSE = sum([(train_pred[i] - trainY[i]) ** 2 for i in range(len(trainY))]) / len(trainY)
train_RMSE = train_MSE ** 0.5
print('Training Error :')
print(f'MAE : {train_MAE}')
print(f'MSE : {train_MSE}')
print(f'RMSE : {train_RMSE}')

Training Error :
MAE : 533.2583331279445
MSE : 519633.2481810662
RMSE : 720.8559136062256


TESTING DATA

In [ ]:
test_MAE = sum([abs(test_pred[i] - testY[i]) for i in range(len(testY))]) / len(testY)
test_MSE = sum([(test_pred[i] - testY[i]) ** 2 for i in range(len(testY))]) / len(testY)
test_RMSE = test_MSE ** 0.5
print('Testing Error :')
print(f'MAE : {test_MAE}')
print(f'MSE : {test_MSE}')
print(f'RMSE : {test_RMSE}')

Testing Error :
MAE : 546.2799015481903
MSE : 497098.1953757691
RMSE : 705.0519097029446


##Analyze training data predictions

In [ ]:
pd.DataFrame({'Actual' : trainY, 'Predicted':train_pred})

,Actual,Predicted
0,4221.85392,4606.326987
1,7609.43808,6503.071349
2,5847.05880,5918.219480
3,4672.50300,4994.327911
4,7883.78688,6613.066425
...,...,...
1939,5830.37964,5678.778979
1940,6110.49600,5870.823228
1941,5289.94368,5445.151226
1942,7962.97392,7536.399077


##Analyze testing data predictions

In [ ]:
pd.DataFrame({'Actual' : testY, 'Predicted':test_pred})

,Actual,Predicted
0,6161.624640,5671.401261
1,5761.792440,6900.126746
2,7390.270800,6397.771184
3,4597.368840,5845.783699
4,5242.712040,5959.132283
...,...,...
211,6764.319072,7487.549637
212,7390.270800,6726.415010
213,6550.077600,6545.325555
214,5722.043040,5567.554283


##PREDICTION

In [ ]:
x = float(input('Enter the previous day load at present hour : '))
x = (x - Xmin) / (Xmax - Xmin)
pred = m * x + c
pred = (pred * (Ymax - Ymin)) + Ymin
print('The Predicted load at present hour = ', pred)

Enter the previous day load at present hour : 5000
The Predicted load at present hour =  5224.7090055152485
